In [25]:
#lib para facilitar o uso de dataframes
import pandas as pd
#lib para facilitar alguns cálculos
import numpy as np
#Lib para auxiliar o processo de count
from collections import Counter

import itertools

In [26]:
#lendo o csv com o dataset pronto
df_voting = pd.read_csv('data/cleaned_voting.csv')

In [27]:
#Confirmando o formato dos dados
df_voting.head()

,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,partido_democrat,partido_republican
0,n,n,n,y,y,y,n,n,n,y,n,y,y,y,n,n,n,y
1,n,n,y,n,n,y,y,y,y,n,y,n,n,n,y,y,y,n
2,y,y,n,n,n,y,y,n,n,n,y,y,n,y,n,y,y,n
3,y,y,y,n,y,y,n,n,n,n,y,n,y,y,n,n,y,n
4,n,n,n,y,y,n,n,n,n,n,n,y,n,y,n,n,n,y


# Regras de associacao

Regras de associacao sao tecnicas de mineiracao de dados usadas para descobrir associacoes interessantes entre atributos de um determinado banco de dados.
A definicao classica de regras de associacao foi apresentada em Agrawal, Imielinski, & Swami (1993) and Han & Kamber, (2006)), e e definida como o seguinte:

Seja T {t1,t2...tn} um conjunto de transacoes e I {i1,i2...in} um conjunto de itens, definimos D como o dado relevante para a tarefa sendo um conjunto de transacoes onde cada T e um conjunto de itens de maneira que T seja um subconjunto de I.

Uma regra de associacao tem a forma X -> Y, onde X e Y sao conjuntos disjuntos

Em nosso dataset as transacoes T representao o conjunto de votos de cada representante, e os Itens representao a reposta ao voto para cada projeto.

O algoritmo aprior é usado para encontrar conjuntos frequentes de items em databases como o nosso.
Ele funciona com a afirmativa que:
> Um sub conjunto de um conjunto frequente provavelmente e trambem um conjunto frequente

Por exemplo:
   se {i1,i2} é um conjunto frequente entao provavelmente {i1} e {i2} tambem devem ser conjuntos de items frequentes
   
- O algoritmo faz varias iteracoes em busca de itens frequentes e usa esses itens frequentes para gerar as regras de associacao

Abaixo vemos a implementacao do algoritmo passo a passo

---

Antes de comecarmos com o algoritmo precisamos realizar a definicao das metricas

In [ ]:
'''
Variavel global para definicao de limite superior para suporte
'''
SUP_THRESHOLD = 0.50

'''
Calculo do suporte
'''
def calculate_support(occurrences, total):
    '''
    suporte = Numero de transacoes que um item aparece
              ---------------------------------------
              Numero total de transacoes
    '''
    return occurrences*1.0/total
'''
Filtra dinamicamente um dataframe para encontrar transacoes que contem as variaveis candidatas
'''
def filter_df(df, candidate_set):
    query = ' & '.join(['{} == "y"'.format(k) for k in candidate_set])
    return df.query(query)
'''
Retorna o suporte de um conjunto de candidatos por meio de um filtro e depois calculo do suporte
'''
def get_support(df, candidate_set):
    filtered_df = filter_df(df, candidate_set)
    support = calculate_support(filtered_df.shape[0], df.shape[0])
    return support
'''
Verifica se o suporte e suficience dado um limiar
'''
def is_support_enough(df, candidate):
    return get_support(df, candidate) >= SUP_THRESHOLD

'''
Filtra os candidatos que nao possuem suporte suficiente
'''
def filter_candidates_by_support(df, candidates):
    return filter(lambda candidate: is_support_enough(df, candidate), candidates)

def get_confianca():
    pass

No bloco a seguir, utilizamos algumas funcoes auxiliares para a execucao do algoritmo

In [129]:
def gera_combinacao(candidatos, k):
    '''
    A partir de uma lista gera todas as combinacoes possiveis de tamanho k
    Ex:
    para k = 2
    F3 = {{1, 2, 3}, {1, 2, 4}, {1, 3, 4}, {1, 3, 5}, {2, 3, 4}}.
    Alguns grupos gerados
    1,2,3 e 1,2,4
    1,2,3 e 1,3,4
    1,2,3 e 1,3,5
    ...
    '''
    #geramos todas as combinacoes possiveis de i para o resto da lista
    #usando a funcao combinations de itertools
    return [list(x) for x in itertools.combinations(candidatos, k)]
'''
Funcao para transformar uma lista de listas em uma lista simples
'''
def unflatten_list(l):
    return [item for sublist in l for item in sublist]

In [111]:
# teste
#filter_candidates([],df_voting)

In [130]:
'''
Funcao utilizada para criar o conjunto de candidatos
'''
def candidate_gen(candidates, k):
    Ck = []
    combinations = gera_combinacao(candidates, k-1)
    # para todo par de itens frequentes
    for f1 in combinations:
        for f2 in combinations:
            # verificamos se o ultimo elemento e diferente
            if f1[-1] < f2[-1]:
                # etapa join
                c = set(unflatten_list(f1) + unflatten_list(f2))
                c = list(c)
                c_subsets = gera_combinacao(c, k-1)
                is_subset = True
                # etapa prune
                for i in c_subsets:
                    if i not in candidates:
                        is_subset = False
                        break
                if is_subset and c not in Ck:
                    Ck.append(c)
            
    
    return Ck

In [137]:
'''
Funcao utilizada para calcular o conjunto inicial L1 {large 1-itemsets}
'''
def init_candidates(df):
    columns = list(df)
    # o primeiro conjunto de candidatos sao as proprias colunas do dataframe
    candidates = [[x] for x in columns]
    # em seguida fazemos um filtro pelo suporte
    candidates = filter_candidates_by_support(df, candidates)
    return candidates

'''
Algoritmo Apriori
'''
def apriori(df):
    res = []
    # calcula o primeiro L1 {large 1-itemsets}
    candidates = init_candidates(df)
    res += candidates
    
    # para cada valor de k entre 2 ate o numero de colunas do dataframe criamos iterativamente novos conjuntos
    for k in range(2, df.shape[1]):
        new_candidates = candidate_gen(candidates, k)
        # caso o novo conjunto tenha zero elementos, paramos o loop
        if len(new_candidates) < 1 :
            break
        
        # filtramos os candidatos com suporte superior ao limiar e adicionamos a nosso conjunto resposta
        # deveriamos parar o loop?
        candidates = filter_candidates_by_support(df, new_candidates)
        if len(candidates) > 0:
            res += candidates
        
    return res
    


In [138]:
frequent_items = apriori(df_voting)
for i in frequent_items:
    print i

['p_2']
['p_3']
['p_6']
['p_7']
['p_8']
['p_9']
['p_10']
['p_13']
['p_14']
['p_16']
['partido_democrat']
['p_3', 'p_8']
['partido_democrat', 'p_3']
['partido_democrat', 'p_8']
['p_6', 'p_14']
['p_3', 'p_16']
['p_7', 'p_16']
['p_8', 'p_16']
['p_16', 'p_9']
['partido_democrat', 'p_16']
['p_3', 'p_8', 'p_16']
['partido_democrat', 'p_3', 'p_16']


In [22]:
#candidate_gen([[1,2,3],[1,2,4],[2,3,4],[1,3,4],[1,3,5]],4)